In [10]:
import os
import shutil
import json
import csv
import glob

# --- CONFIGURATION ---
BASE_DIR = r'/Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Drowsiness/S5'
FOLDERS_TO_MERGE = ["0", "1", "2", "3"]
OUTPUT_FOLDER_NAME = "frames"

# STRICT TARGET: We only want this exact string
TARGET_TYPE = "blinks/blinking"

def get_blinking_frames(json_path):
    """
    Parses the JSON and filters strictly for TARGET_TYPE.
    Logs what it finds and what it skips.
    """
    blink_frames = set()
    found_types = set()

    try:
        with open(json_path, 'r') as f:
            data = json.load(f)

        # Navigate to actions
        openlabel = data.get("openlabel", {})
        actions = openlabel.get("actions", {})

        print(f"  [Analysis] Scanning Action Types in {os.path.basename(json_path)}:")

        for action_id, content in actions.items():
            action_type = content.get("type")
            found_types.add(action_type)

            # STRICT FILTER
            if action_type == TARGET_TYPE:
                print(f"    -> MATCH: Found Target '{action_type}' (ID: {action_id}). Extracting frames...")

                intervals = content.get("frame_intervals", [])
                for interval in intervals:
                    start = interval.get("frame_start")
                    end = interval.get("frame_end")

                    if start is not None and end is not None:
                        # Add frames (inclusive)
                        for frame_num in range(start, end + 1):
                            blink_frames.add(frame_num)
            else:
                # Log what we are ignoring so you can be sure
                pass
                # Uncomment the next line if you want to see every skipped type
                # print(f"    -> SKIP:  Ignoring '{action_type}' (ID: {action_id})")

        # Summary for the user
        if TARGET_TYPE not in found_types:
            print(f"  [WARNING] Target '{TARGET_TYPE}' was NOT found in this file!")
            print(f"  [Available Types]: {list(found_types)}")

    except Exception as e:
        print(f"  [Error] Failed to parse {os.path.basename(json_path)}: {e}")

    return blink_frames

def process_subject(subject_name):
    subject_dir = os.path.join(BASE_DIR, subject_name)
    target_dir = os.path.join(subject_dir, OUTPUT_FOLDER_NAME)

    # 1. Find JSON
    json_files = glob.glob(os.path.join(subject_dir, "*.json"))
    if not json_files:
        return

    print(f"\nProcessing Subject: {subject_name}...")
    json_path = json_files[0]

    # 2. Get Labels (STRICT MODE)
    blinking_frames = get_blinking_frames(json_path)
    print(f"  Total blinking frames captured: {len(blinking_frames)}")

    # 3. Create Output Folder
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    labels = []
    processed_count = 0

    # 4. Merge Images & Generate Labels
    for sub in FOLDERS_TO_MERGE:
        source_sub = os.path.join(subject_dir, sub)
        if not os.path.exists(source_sub):
            continue

        files = [f for f in os.listdir(source_sub) if f.lower().endswith('.jpg')]
        files.sort()

        for filename in files:
            src_path = os.path.join(source_sub, filename)
            dst_path = os.path.join(target_dir, filename)

            # Copy file
            if not os.path.exists(dst_path):
                shutil.copy2(src_path, dst_path)

            # Generate Label
            try:
                frame_num = int(os.path.splitext(filename)[0])

                # BINARY LABELING: 1 = Blinking, 0 = Anything else (Open, closing, etc.)
                if frame_num in blinking_frames:
                    val = 1
                    type_str = "Blinking"
                else:
                    val = 0
                    type_str = "Not-Blinking"

                labels.append([filename, val, type_str])
                processed_count += 1
            except ValueError:
                pass

    # 5. Save CSV
    if labels:
        csv_path = os.path.join(target_dir, "labels.csv")
        with open(csv_path, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["filename", "label", "type"])
            writer.writerows(labels)
        print(f"  [Success] Saved to {subject_name}/{OUTPUT_FOLDER_NAME}/labels.csv")

def main():
    print(f"Root Directory: {BASE_DIR}")
    print(f"Targeting ONLY: '{TARGET_TYPE}' (Ignoring 'eyes_state/opening')")

    all_items = os.listdir(BASE_DIR)
    all_items.sort()

    for item in all_items:
        item_path = os.path.join(BASE_DIR, item)
        if os.path.isdir(item_path) and item.isdigit():
            process_subject(item)

    print("\nAll processing complete.")

if __name__ == "__main__":
    main()

Root Directory: /Users/shikharsrivastava/Desktop/Thesis/Thesis/MultiTask/Drowsiness/S5
Targeting ONLY: 'blinks/blinking' (Ignoring 'eyes_state/opening')

Processing Subject: 01...
  [Analysis] Scanning Action Types in gA_1_s5_2019-03-14T14;26;17+01;00_rgb_ann_drowsiness.json:
    -> MATCH: Found Target 'blinks/blinking' (ID: 4). Extracting frames...
  Total blinking frames captured: 1884
  [Success] Saved to 01/frames/labels.csv

Processing Subject: 02...
  [Analysis] Scanning Action Types in gA_2_s5_2019-03-13T09;19;23+01;00_rgb_ann_drowsiness.json:
    -> MATCH: Found Target 'blinks/blinking' (ID: 4). Extracting frames...
  Total blinking frames captured: 1630

Processing Subject: 03...
  [Analysis] Scanning Action Types in gA_3_s5_2019-03-13T09;36;25+01;00_rgb_ann_drowsiness.json:
    -> MATCH: Found Target 'blinks/blinking' (ID: 4). Extracting frames...
  Total blinking frames captured: 2488

Processing Subject: 04...
  [Analysis] Scanning Action Types in gA_4_s5_2019-03-13T10;56;5